In [1]:
import numpy as np
import pandas as pd

### 第四章 分组
#### 一. 分组模式及其对象
##### 1.分组的一般模式
分组操作在日常生活中使用极其广泛，例如：         
- 依据性别分组，统计全国人口寿命的平均值
- 依据季节分组，对每一个季节的温度进行组内标准化
- 依据班级筛选出组内数学分数的平均值超过80分的班级               
从上述的几个例子中不难看出，想要实现分组操作，必须明确三个要素：分组依据、数据来源、操作及其返回结果。    
同时从充分性的角度来说，如果明确了这三方面，就能确定一个分组操作，从而分组代码的一般模式即：

df.groupby(分组依据)[数据来源].使用操作

例如第一个例子中的代码就应该如下：

df.groupby('Gender')['Longevity'].mean()

现在返回到学生体测的数据集上，如果想要按照性别统计身高中位数，就可以如下写出：

In [2]:
df = pd.read_csv('../3_index/ch3/learn_pandas.csv')
df.groupby('Gender')['Height'].median()

Gender
Female    159.6
Male      173.4
Name: Height, dtype: float64

##### 2.分组依据的本质
前面提到的若干例子都是以单一维度进行分组的，比如根据性别，如果现在需要根据多个维度进行分组，该如何做？事实上，只需在groupby中传入相应列名构成的列表即可。例如，现希望根据学校和性别进行分组，统计身高的均值就可以如下写出：

In [3]:
df.groupby(['School', 'Gender'])['Height'].mean()

School  Gender
A       Female    159.122500
        Male      176.760000
B       Female    158.666667
        Male      172.030000
C       Female    158.776923
        Male      174.212500
D       Female    159.753333
        Male      171.638889
Name: Height, dtype: float64

目前为止，groupby的分组依据都是直接可以从列中按照名字获取的，那如果希望通过一定的复杂逻辑来分组，例如根据学生体重是否超过总体均值来分组，同样还是计算身高的均值。       
首先应该先写出分组条件：

In [4]:
condition = df.Weight > df.Weight.mean()

然后将其传入`groupby`中：

In [5]:
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

练一练    
请根据上下四分位数分割，将体重分为high、normal、low三组，统计身高的均值.

In [6]:
# q1 = df['Weight'].quantile(0.25)
# q3 = df['Weight'].quantile(0.75)
# def weight_group(weight):
#     if weight < q1:
#         return 'low'
#     elif weight > q3:
#         return 'high'
#     else:
#         return 'normal'
    
# df['WeightGroup'] = df['Weight'].apply(weight_group)
# df.groupby('WeightGroup')['Height'].mean()

从索引可以看出，其实最后产生的结果就是按照条件列表中元素的值（此处是True和False）来分组，下面用随机传入字母序列来验证这一想法：

In [7]:
item = np.random.choice(list('abc'), df.shape[0])
df.groupby(item)['Height'].mean()

a    163.248571
b    164.280328
c    161.930769
Name: Height, dtype: float64

此处的索引就是原先item中的元素，如果传入多个序列进入groupby，那么最后分组的依据就是这两个序列对应行的唯一组合：

In [8]:
df.groupby([condition, item])['Height'].mean()

Weight   
False   a    158.204255
        b    160.179070
        c    158.759459
True    a    173.556522
        b    174.077778
        c    169.753333
Name: Height, dtype: float64

由此可以看出，之前传入列名只是一种简便的记号，事实上等价于传入的是一个或多个列，最后分组的依据来自于数据来源组合的unique值，通过drop_duplicates就能知道具体的组类别：

In [9]:
df[['School', 'Gender']].drop_duplicates()

,School,Gender
0,A,Female
1,B,Male
2,A,Male
3,C,Female
4,C,Male
5,D,Female
9,B,Female
16,D,Male


In [10]:
df.groupby([df['School'],df['Gender']])['Height'].mean()

School  Gender
A       Female    159.122500
        Male      176.760000
B       Female    158.666667
        Male      172.030000
C       Female    158.776923
        Male      174.212500
D       Female    159.753333
        Male      171.638889
Name: Height, dtype: float64

##### 3.Groupby对象
能够注意到,最终具体做分组操作时，所调用的方法都来自于`pandas`中的`groupby`对象,这个对象上定义了许多方法，也具有一些方便的属性.

In [11]:
gb = df.groupby(['School', 'Grade'])
gb

通过`ngroups`属性，可以得到分组个数：

In [12]:
gb.ngroups

16

通过`groups`属性，可以返回从组名映射到组索引列表的字典:

In [13]:
res = gb.groups
res.keys() # 字典的值由于是索引，元素个数过多，此处只展示字典的键

dict_keys([('A', 'Freshman'), ('A', 'Junior'), ('A', 'Senior'), ('A', 'Sophomore'), ('B', 'Freshman'), ('B', 'Junior'), ('B', 'Senior'), ('B', 'Sophomore'), ('C', 'Freshman'), ('C', 'Junior'), ('C', 'Senior'), ('C', 'Sophomore'), ('D', 'Freshman'), ('D', 'Junior'), ('D', 'Senior'), ('D', 'Sophomore')])

当`size`作为`DataFrame`的属性时，返回的是表长乘以表宽的大小，但在`groupby`对象上表示统计每个组的元素个数：

In [14]:
gb.size()

School  Grade    
A       Freshman     13
        Junior       17
        Senior       22
        Sophomore     5
B       Freshman     13
        Junior        8
        Senior        8
        Sophomore     5
C       Freshman      9
        Junior       12
        Senior       11
        Sophomore     8
D       Freshman     17
        Junior       22
        Senior       14
        Sophomore    16
dtype: int64

通过`get_group`方法可以直接获取所在组对应的行，此时必须知道组的具体名字：

In [15]:
gb.get_group(('A','Freshman'))

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,A,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
6,A,Freshman,Qiang Chu,Female,162.5,52.0,N,1,2019/12/12,0:03:53
10,A,Freshman,Xiaopeng Zhou,Male,174.1,74.0,N,1,2019/9/29,0:05:16
60,A,Freshman,Yanpeng Lv,Male,NaN,65.0,N,1,2019/11/17,0:04:13
114,A,Freshman,Xiaopeng Zhao,Female,161.0,53.0,N,3,2019/9/25,0:05:13
117,A,Freshman,Chunli Zhao,Male,180.2,83.0,N,1,2020/1/7,0:04:33
119,A,Freshman,Peng Zhang,Female,163.1,NaN,N,3,2019/9/23,0:04:31
121,A,Freshman,Xiaoquan Sun,Female,154.6,40.0,N,3,2019/11/12,0:04:05
141,A,Freshman,Chunmei Shi,Female,164.9,52.0,N,1,2019/9/8,0:03:33
148,A,Freshman,Xiaomei Yang,Female,159.3,49.0,N,1,2019/9/17,0:04:22


这里列出了2个属性和2个方法，而先前的mean、median都是groupby对象上的方法，这些函数和许多其他函数的操作具有高度相似性，将在之后的小节进行专门介绍。

##### 4.分组的三大操作
熟悉了一些分组的基本知识后，重新回到开头举的三个例子，可能会发现一些端倪，即这三种类型分组返回的数据型态并不一样：       
- 第一个例子中，每一个组返回一个标量值，可以是平均值，中位数，组容量`size`等
- 第二个例子中，做了原序列的标准化处理，也就是说每组返回的是一个`Series`类型
- 第三个例子中，既不是标量也不是序列，返回的整个组所在行的本身，即返回了`DataFrame`类型         
由此，引申出分组的三大操作：聚合、变换和过滤，分别对应了三个例子的操作，下面就要分别介绍相应的`agg`、`transform`和`filter`函数及其操作

#### 二.聚合函数
##### 1.内置聚合函数
在介绍agg之前，首先要了解一些直接定义在groupby对象的聚合函数，因为它的速度基本都会经过内部的优化，使用功能时应当优先考虑。根据返回标量值的原则，包括如下函数：`max/min/mean/median/count/all/any/idxmax/idxmin/mad/nunique/skew/quantile/sum/std/var/sem/size/prod`。

In [16]:
gb = df.groupby('Gender')['Height']
gb.idxmin()

Gender
Female    143
Male      199
Name: Height, dtype: int64

In [17]:
gb.quantile(0.95)

Gender
Female    166.8
Male      185.9
Name: Height, dtype: float64

这些聚合函数当传入的数据来源包含多个列时，将按照列进行迭代计算：

In [18]:
gb = df.groupby('Gender')[['Height', 'Weight']]
gb.max()

,Height,Weight
Gender,,
Female,170.2,63.0
Male,193.9,89.0


##### 2.agg方法
虽然在`groupby`对象上定义了许多方便的函数，但仍然有以下不便之处：
- 无法同时使用多个函数
- 无法对特定的列使用特定的聚合函数
- 无法使用自定义的聚合函数
- 无法直接对结果的列名在聚合前进行自定义命名     
下面说明如何通过`agg`函数解决这四类问题：        
【a】使用多个函数         
当使用多个聚合函数时，需要用列表的形式把内置聚合函数对应的字符串传入，先前提到的所有字符串都是合法的。

In [19]:
gb.agg(['sum', 'idxmax', 'skew'])

Height                   Weight                 
            sum idxmax      skew     sum idxmax      skew
Gender                                                   
Female  21014.0     28 -0.219253  6469.0     28 -0.268482
Male     8854.9    193  0.437535  3929.0      2 -0.332393

从结果看，此时的列索引为多级索引，第一层为数据源，第二层为使用的聚合方法，分别逐一对列使用聚合，因此结果为6列。          
【b】对特定的列使用特定的聚合函数       
对于方法和列的特殊对应，可以通过构造字典传入`agg`中实现，其中字典以列名为键，以聚合字符串或字符串列表为值。

In [20]:
gb.agg({'Height': ['mean', 'max'],'Weight': 'count'})

Height        Weight
             mean    max  count
Gender                         
Female  159.19697  170.2    135
Male    173.62549  193.9     54

【c】使用自定义函数         
在agg中可以使用具体的自定义函数，需要注意传入函数的参数是之前数据源中的列，逐列进行计算。下面分组计算身高和体重的极差：

In [22]:
gb.agg(lambda x: x.mean() - x.min())

,Height,Weight
Gender,,
Female,13.79697,13.918519
Male,17.92549,21.759259


由于传入的是序列，因此序列上的方法和属性都是可以在函数中使用的，只需保证返回值是标量即可。下面的例子是指，如果组的指标均值，超过该指标的总体均值，返回High，否则返回Low。

In [25]:
def my_func(s):
    res = 'High'
    if s.mean() <= df[s.name].mean():
        res = 'Low'
    return res
gb.agg(my_func)

,Height,Weight
Gender,,
Female,Low,Low
Male,High,High


【d】聚合结果重命名         
如果想要对聚合结果的列名进行重命名，只需要将上述函数的位置改写成元组，元组的第一个元素为新的名字，第二个位置为原来的函数，包括聚合字符串和自定义函数，现举若干例子说明：

In [26]:
gb.agg([('range', lambda x: x.max() - x.min()), ('my_sum', 'sum')])

Height          Weight        
        range   my_sum  range  my_sum
Gender                               
Female   24.8  21014.0   29.0  6469.0
Male     38.2   8854.9   38.0  3929.0

In [27]:
gb.agg({'Height': [('my_func', my_func), 'sum'],'Weight': lambda x: x.max()})

Height            Weight
       my_func      sum <lambda>
Gender                          
Female     Low  21014.0     63.0
Male      High   8854.9     89.0

另外需要注意，使用对一个或者多个列使用单个聚合的时候，重命名需要加方括号，否则就不知道是新的名字还是手误输错的内置函数字符串：

In [28]:
gb.agg([('my_sum', 'sum')])

,Height,Weight
,my_sum,my_sum
Gender,,
Female,21014.0,6469.0
Male,8854.9,3929.0


In [30]:
gb.agg({'Height': [('my_func', my_func), 'sum'], 'Weight': [('range', lambda x: x.max() - x.min())]})

Height          Weight
       my_func      sum  range
Gender                        
Female     Low  21014.0   29.0
Male      High   8854.9   38.0

#### 三.变换和过滤
##### 1.变换函数与transform方法
变换函数的返回值为同长度的序列，最常用的内置变换函数是累计函数：`cumcount/cumsum/cumprod/cummax/cummin`，它们的使用方式和聚合函数类似，只不过完成的是组内累计操作。此外在`groupby`对象上还定义了填充类和滑窗类的变换函数，这些函数的一般形式将会分别在第七章和第十章中讨论，此处略过。

In [31]:
gb.cummax().head()

,Height,Weight
0,158.9,46.0
1,166.5,70.0
2,188.9,89.0
3,NaN,46.0
4,188.9,89.0


当用自定义变换时需要使用transform方法，被调用的自定义函数，其传入值为数据源的序列，与`agg`的传入类型是一致的，其最后的返回结果是行列索引与数据源一致的`DataFrame`.         
现对身高和体重进行分组标准化，即减去组均值后除以组的标准差：

In [32]:
gb.transform(lambda x: (x - x.mean()) / x.std()).head()

,Height,Weight
0,-0.058760,-0.354888
1,-1.010925,-0.355000
2,2.167063,2.089498
3,NaN,-1.279789
4,0.053133,0.159631


In [41]:
# 对于transform方法无法像agg一样，通过传入字典来对指定列使用特定的变换，如果需要在一次transform的调用中实现这种功能，解决方案如下：
mapping = {
    "Height": lambda s: (s - s.mean()) / s.std(ddof=0),  # zscore
    "Weight": lambda s: s / s.sum(),                     # 比例
}
gb.transform(lambda d: mapping.get(d.name, lambda x: x)(d)).head()

,Height,Weight
0,-0.058983,0.007111
1,-1.020984,0.017816
2,2.188626,0.022652
3,NaN,0.006338
4,0.053662,0.018834


前面提到了`transform`只能返回同长度的序列，但事实上还可以返回一个标量，这会使得结果被广播到其所在的整个组，这种标量广播的技巧在特征工程中是非常常见的。例如，构造两列新特征来分别表示样本所在性别组的身高均值和体重均值:

In [42]:
gb.transform('mean').head() # 传入返回标量的函数也是可以的

,Height,Weight
0,159.19697,47.918519
1,173.62549,72.759259
2,173.62549,72.759259
3,159.19697,47.918519
4,173.62549,72.759259


##### 2.组索引与过滤
在上一章中介绍了索引的用法，那么索引和过滤有什么区别呢？     
过滤在分组中是对于组的过滤，而索引是对于行的过滤，在第二章中的返回值，无论是布尔列表还是元素列表或者位置列表，本质上都是对于行的筛选，即如果符合筛选条件的则选入结果表，否则不选入。     
组过滤作为行过滤的推广，指的是如果对一个组的全体所在行进行统计的结果返回True则会被保留，False则该组会被过滤，最后把所有未被过滤的组其对应的所在行拼接起来作为DataFrame返回。       
在`groupby`对象中，定义了`filter`方法进行组的筛选，其中自定义函数的输入参数为数据源构成的`DataFrame`本身，在之前例子中定义的`groupby`对象中，传入的就是`df[['Height', 'Weight']]`，因此所有表方法和属性都可以在自定义函数中相应地使用，同时只需保证自定义函数的返回为布尔值即可。       
例如，在原表中通过过滤得到所有容量大于100的组：

In [49]:
gb.filter(lambda x : x.shape[0] > 100)

,Height,Weight
0,158.9,46.0
3,NaN,41.0
5,158.0,51.0
6,162.5,52.0
7,161.9,50.0
...,...,...
191,166.6,54.0
194,160.3,49.0
195,153.9,46.0
196,160.9,50.0


#### 四.跨列分组
##### 1.apply的引入
之前几节介绍了三大分组操作，但事实上还有一种常见的分组场景，无法用前面介绍的任何一种方法处理，例如现在如下定义身体质量指数BMI： $BMI = Weight/Height$ 
其中体重和身高的单位分别为千克和米，需要分组计算组BMI的均值。          
首先，这显然不是过滤操作，因此`filter`不符合要求；其次，返回的均值是标量而不是序列，因此`transform`不符合要求；最后，似乎使用`agg`函数能够处理，但是之前强调过聚合函数是逐列处理的，而不能够
多列数据同时处理。由此，引出了`apply`函数来解决这一问题。

##### 2.apply的使用
在设计上，`apply`的自定义函数传入参数与`filter`完全一致,只不过后者只允许返回布尔值。现如下解决上述计算问题:

In [50]:
def BMI(x):
    Height = x['Height'] / 100  # 转换为米
    Weight = x['Weight']
    BMI_value = Weight / (Height ** 2)
    return BMI_value.mean()

gb.apply(BMI)

Gender
Female    18.860930
Male      24.318654
dtype: float64

除了返回标量之外，`apply`方法还可以返回一维`Series`和二维`DataFrame`，但它们产生的数据框维数和多级索引的层数应当如何变化？        
下面举三组例子就非常容易明白结果是如何生成的：            
【a】标量情况：结果得到的是`Series`,索引与`agg`的结果一致

In [51]:
gb = df.groupby(['Gender','Test_Number'])[['Height','Weight']]
gb.apply(lambda x: 0)

Gender  Test_Number
Female  1              0
        2              0
        3              0
Male    1              0
        2              0
        3              0
dtype: int64

In [53]:
gb.apply(lambda x: [0,0]) # 虽然是列表，但是作为返回值仍然看作标量

Gender  Test_Number
Female  1              [0, 0]
        2              [0, 0]
        3              [0, 0]
Male    1              [0, 0]
        2              [0, 0]
        3              [0, 0]
dtype: object

【b】`Series`情况：得到的是`DataFrame`,行索引与标量情况一致，列索引为`Series`的索引

In [54]:
gb.apply(lambda x: pd.Series([0,0], index = ['a','b']))

a  b
Gender Test_Number      
Female 1            0  0
       2            0  0
       3            0  0
Male   1            0  0
       2            0  0
       3            0  0

【c】`DataFrame`情况：得到的是`DataFrame`，行索引最内层在每个组原先`agg`的结果索引上，再加一层返回的`DataFrame`行索引,同时分组结果`DataFrame`的列索引和返回的`DataFrame`列索引一致。

In [55]:
gb.apply(lambda x: pd.DataFrame(np.ones((2,2)), index = ['a','b'], columns = pd.Index([('w','x'), ('y','z')])))

w    y
                        x    z
Gender Test_Number            
Female 1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0
Male   1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0

最后需要强调的是，`apply`函数的灵活性是以牺牲一定性能为代价换得的，除非需要使用跨列处理的分组处理，否则应当使用其他专门设计的`groupby`对象方法，否则在性能上会存在较大的差距。同时，在使用聚合函数和变换函数时，也应当优先使用内置函数，它们经过了高度的性能优化，一般而言在速度上都会快于用自定义函数来实现。

#### 五.练习
##### Ex1：汽车数据集
现有一份汽车数据集，其中`Brand,Disp.,HP`分别代表汽车品牌、发动机蓄量、发动机输出。

In [61]:
df = pd.read_csv('car.csv')
df.head(3)

,Brand,Price,Country,Reliability,Mileage,Type,Weight,Disp.,HP
0,Eagle Summit 4,8895,USA,4.0,33,Small,2560,97,113
1,Ford Escort 4,7402,USA,2.0,33,Small,2345,114,90
2,Ford Festiva 4,6319,Korea,4.0,37,Small,1845,81,63


1. 先过滤出所属Country数超过2个的汽车，即若该汽车的Country在总体数据集中出现次数不超过2则剔除，再按Country分组计算价格均值、价格变异系数、该Country的汽车数量，其中变异系数的计算方法是标准差除以均值，并在结果中把变异系数重命名为CoV。

In [67]:
d1 = df.groupby('Country').filter(lambda x: x.shape[0] > 2)
d1.groupby('Country').agg({'Price': ['mean',('CoV', lambda x: x.std() / x.mean()),'count']},)

Price                
                   mean       CoV count
Country                                
Japan      13938.052632  0.387429    19
Japan/USA  10067.571429  0.240040     7
Korea       7857.333333  0.243435     3
USA        12543.269231  0.203344    26

2. 按照表中位置的前三分之一、中间三分之一和后三分之一分组，统计Price的均值。

In [68]:
df.shape[0]

60

In [69]:
condition = ['Head']*20 + ['Middle']*20 + ['Tail']*20
df.groupby(condition)['Price'].mean()

Head       9069.95
Middle    13356.40
Tail      15420.65
Name: Price, dtype: float64

3. 对类型Type分组，对Price和HP分别计算最大值和最小值，结果会产生多级索引，请用下划线把多级列索引合并为单层索引。

In [72]:
res = df.groupby('Type').agg({'Price': ['max'], 'HP': ['min']})
res.columns = res.columns.map(lambda x: '_'.join(x))
res

,Price_max,HP_min
Type,,
Compact,18900,95
Large,17257,150
Medium,24760,110
Small,9995,63
Sporty,13945,92
Van,15395,106


4. 对类型Type分组，对HP进行组内的min-max归一化。

In [74]:
df.groupby('Type')['HP'].transform(lambda x: (x - x.min()) / (x.max() - x.min())).head()

0    1.00
1    0.54
2    0.00
3    0.58
4    0.80
Name: HP, dtype: float64

5. 对类型Type分组，计算Disp.与HP的相关系数。

In [76]:
df.groupby('Type')[['Disp.','HP']].apply(lambda x: np.corrcoef(x['Disp.'], x['HP'])[0,1])

Type
Compact    0.586087
Large     -0.242765
Medium     0.370491
Small      0.603916
Sporty     0.871426
Van        0.819881
dtype: float64

##### Ex2:实现transform函数

In [77]:
class my_groupby:
    def __init__(self, my_df, group_cols):
        self.my_df = my_df.copy()
        self.groups = my_df[group_cols].drop_duplicates()
        if isinstance(self.groups, pd.Series):
            self.groups = self.groups.to_frame()
        self.group_cols = self.groups.columns.tolist()
        self.groups = {i: self.groups[i].values.tolist() for i in self.groups.columns}
        self.transform_col = None
    def __getitem__(self, col):
        self.pr_col = [col] if isinstance(col, str) else list(col)
        return self
    def transform(self, my_func):
        self.num = len(self.groups[self.group_cols[0]])
        L_order, L_value = np.array([]), np.array([])
        for i in range(self.num):
            group_df = self.my_df.reset_index().copy()
            for col in self.group_cols:
                group_df = group_df[group_df[col]==self.groups[col][i]]
            group_df = group_df[self.pr_col]
            if group_df.shape[1] == 1:
                group_df = group_df.iloc[:, 0]
            group_res = my_func(group_df)
            if not isinstance(group_res, pd.Series):
                group_res = pd.Series(group_res,index=group_df.index,name=group_df.name)
            L_order = np.r_[L_order, group_res.index]
            L_value = np.r_[L_value, group_res.values]
        self.res = pd.Series(pd.Series(L_value, index=L_order).sort_index().values,index=self.my_df.reset_index().index, name=my_func.__name__)
        return self.res

my_groupby(df, 'Type')

In [78]:
# 单列分组
def f(s):
    res = (s-s.min())/(s.max()-s.min())
    return res
my_groupby(df, 'Type')['Price'].transform(f).head()

0    0.733592
1    0.372003
2    0.109712
3    0.186244
4    0.177525
Name: f, dtype: float64

In [79]:
df.groupby('Type')['Price'].transform(f).head()

0    0.733592
1    0.372003
2    0.109712
3    0.186244
4    0.177525
Name: Price, dtype: float64

In [81]:
# 多列分组
my_groupby(df, ['Type','Country'])['Price'].transform(f).head()

0    1.000000
1    0.000000
2    0.000000
3    0.000000
4    0.196357
Name: f, dtype: float64

In [82]:
df.groupby(['Type','Country'])['Price'].transform(f).head()

0    1.000000
1    0.000000
2    0.000000
3    0.000000
4    0.196357
Name: Price, dtype: float64

In [83]:
# 标量广播
my_groupby(df, 'Type')['Price'].transform(lambda x:x.mean()).head()

0    7682.384615
1    7682.384615
2    7682.384615
3    7682.384615
4    7682.384615
Name: <lambda>, dtype: float64

In [84]:
df.groupby('Type')['Price'].transform(lambda x:x.mean()).head()

0    7682.384615
1    7682.384615
2    7682.384615
3    7682.384615
4    7682.384615
Name: Price, dtype: float64

In [85]:
# 跨列计算
my_groupby(df, 'Type')['Disp.', 'HP'].transform(lambda x: x['Disp.']/x.HP).head()

0    0.858407
1    1.266667
2    1.285714
3    0.989130
4    1.097087
Name: <lambda>, dtype: float64